In [1]:
# %%
import os
import sys

from matplotlib import animation
sys.path.append("/home/mbortkie/repos/crl_subgoal/src")
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# %%
import functools
import os
import distrax


import jax
import jax.numpy as jnp
from jax import random
from typing import Tuple, Dict, Any
from dataclasses import dataclass
import chex
from flax import struct
from absl import app, flags
from ml_collections import config_flags
from impls.agents import agents
from config import SRC_ROOT_DIR
from envs.block_moving_env import *
from train import *
from impls.utils.checkpoints import restore_agent, save_agent
from config import Config, ExpConfig
from envs import legal_envs
import matplotlib.pyplot as plt
from impls.utils.networks import GCDiscreteActor
import copy
import numpy as np




In [2]:
# %%
RANGE_GENERALIZATION = [1,2,3,4,5,6,7,9,11]
EPISODE_LENGTH = 100
NUM_ENVS = 1024
CHECKPOINT = 50
RUN_NAME = f"CORRECTLY_{CHECKPOINT}_ckpt_short_more_data"
MODEL_PATH = "/home/mbortkie/repos/crl_subgoal/experiments/test_generalization_sc_20250820_001752/runs/CORRECTLY_fixed_bug_with_solved_state_moving_boxes_5_grid_5_range_3_7_alpha_0.1"
EPOCHS = 101
EVAL_EVERY = 10
FIGURES_PATH = f"/home/mbortkie/repos/crl_subgoal/notebooks/figures/{RUN_NAME}"
GIF_PATH = f"{FIGURES_PATH}/gifs"
os.makedirs(FIGURES_PATH, exist_ok=True)
os.makedirs(GIF_PATH, exist_ok=True)


In [3]:
# %%
config = Config(
    exp=ExpConfig(seed=0, name="test"),
    env=BoxPushingConfig(
        grid_size=5,
        number_of_boxes_min=3,
        number_of_boxes_max=7,
        number_of_moving_boxes_max=5
    )
)

# %%
env = create_env(config.env)
env = AutoResetWrapper(env)
key = random.PRNGKey(config.exp.seed)
env.step = jax.jit(jax.vmap(env.step))
env.reset = jax.jit(jax.vmap(env.reset))
partial_flatten = functools.partial(flatten_batch, get_next_obs=config.agent.use_next_obs)
jitted_flatten_batch = jax.jit(jax.vmap(partial_flatten, in_axes=(None, 0, 0)), static_argnums=(0,))
dummy_timestep = env.get_dummy_timestep(key)


In [4]:
replay_buffer = jit_wrap(
    TrajectoryUniformSamplingQueue(
        max_replay_size=config.exp.max_replay_size,
        dummy_data_sample=dummy_timestep,
        sample_batch_size=config.exp.batch_size,
        num_envs=config.exp.num_envs,
        episode_length=config.env.episode_length,
    )
)
buffer_state = jax.jit(replay_buffer.init)(key)

example_batch = {
    'observations':dummy_timestep.grid.reshape(1, -1),  # Add batch dimension 
    'next_observations': dummy_timestep.grid.reshape(1, -1),
    'actions': jnp.ones((1,), dtype=jnp.int8) * (env._env.action_space-1), # TODO: make sure it should be the maximal value of action space  # Single action for batch size 1
    'rewards': dummy_timestep.reward.reshape(1, -1),
    'masks': 1.0 - dummy_timestep.reward.reshape(1, -1), 
    'value_goals': dummy_timestep.grid.reshape(1, -1),
    'actor_goals': dummy_timestep.grid.reshape(1, -1),
}

# %%
agent, config = restore_agent(example_batch, MODEL_PATH, CHECKPOINT)

# %%
keys = random.split(random.PRNGKey(0), NUM_ENVS)
state, info = env.reset(keys)

# %%
dummy_timestep = env.get_dummy_timestep(key)

replay_buffer = jit_wrap(
    TrajectoryUniformSamplingQueue(
        max_replay_size=config.exp.max_replay_size,
        dummy_data_sample=dummy_timestep,
        sample_batch_size=config.exp.batch_size,
        num_envs=config.exp.num_envs,
        episode_length=config.env.episode_length,
    )
)
buffer_state = jax.jit(replay_buffer.init)(key)

Restored from /home/mbortkie/repos/crl_subgoal/experiments/test_generalization_sc_20250820_001752/runs/CORRECTLY_fixed_bug_with_solved_state_moving_boxes_5_grid_5_range_3_7_alpha_0.1/params_50.pkl


In [5]:
@jax.jit
def make_batch(buffer_state, key):
    key, batch_key, double_batch_key = jax.random.split(key, 3)
    # Sample and process transitions
    buffer_state, transitions = replay_buffer.sample(buffer_state)
    batch_keys = jax.random.split(batch_key, transitions.grid.shape[0])
    state, next_state, future_state, goal_index = jitted_flatten_batch(config.exp.gamma, transitions, batch_keys)

    state, actions, future_state, goal_index = get_single_pair_from_every_env(state, future_state, goal_index, double_batch_key, use_double_batch_trick=config.exp.use_double_batch_trick)
    if not config.exp.use_targets:
        state = state.replace(grid=GridStatesEnum.remove_targets(state.grid))
        future_state = future_state.replace(grid=GridStatesEnum.remove_targets(future_state.grid))
    # Create valid batch
    batch = {
        'observations': state.grid.reshape(state.grid.shape[0], -1),
        'next_observations': future_state.grid.reshape(future_state.grid.shape[0], -1),
        'actions': actions.squeeze(),
        'rewards': state.reward.reshape(state.reward.shape[0], -1),
        'masks': 1.0 - state.reward.reshape(state.reward.shape[0], -1), # TODO: add success and reward separately
        'value_goals': future_state.grid.reshape(future_state.grid.shape[0], -1),
        'actor_goals': future_state.grid.reshape(future_state.grid.shape[0], -1),
    }
    return buffer_state, batch

def value_transform(x):
    return jnp.log(jnp.maximum(x, 1e-6))

# Data creation

In [6]:
data_key = random.PRNGKey(0)
_, _, timesteps = collect_data(agent, data_key, env, config.exp.num_envs, config.env.episode_length, use_targets=config.exp.use_targets)
buffer_state = replay_buffer.insert(buffer_state, timesteps)

2025-08-22 16:27:43.185657: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.


In [12]:
batch_keys = jax.random.split(data_key, config.exp.num_envs)

In [13]:
batch_keys.shape

(1024, 2)

In [14]:
timesteps = jax.tree_util.tree_map(lambda x: x.swapaxes(1, 0), timesteps)
timesteps.grid.shape

(1024, 100, 5, 5)

In [ ]:
state, next_state, future_state, goal_index = jitted_flatten_batch(config.exp.gamma, timesteps, batch_keys)

In [17]:
state.grid.shape, next_state.grid.shape, future_state.grid.shape

((1024, 99, 5, 5), (1024, 99, 5, 5), (1024, 99, 5, 5))

In [18]:
for i in range(30):
    print("________________")
    print(state.grid[0, i])
    print(next_state.grid[0, i])
    print(future_state.grid[0, i])

________________
[[ 0 10  2  0  0]
 [ 2  1  1  0  0]
 [ 6  0  0  0  0]
 [ 2  0  1  2  0]
 [ 0  0  0  1  1]]
[[ 0 10  2  0  0]
 [ 2  1  1  0  0]
 [ 2  3  0  0  0]
 [ 2  0  1  2  0]
 [ 0  0  0  1  1]]
[[ 0 10 10  0  0]
 [10  0  0  0  0]
 [ 8  0  0  0  0]
 [10  0  0 10  0]
 [ 0  0  0  0  0]]
________________
[[ 0 10  2  0  0]
 [ 2  1  1  0  0]
 [ 2  3  0  0  0]
 [ 2  0  1  2  0]
 [ 0  0  0  1  1]]
[[ 0 10  2  0  0]
 [ 2  1  1  0  0]
 [ 2  0  3  0  0]
 [ 2  0  1  2  0]
 [ 0  0  0  1  1]]
[[ 0 10  2  0  0]
 [ 2  1  1  0  0]
 [ 2  0  0  3  0]
 [ 2  0  1  2  0]
 [ 0  0  0  1  1]]
________________
[[ 0 10  2  0  0]
 [ 2  1  1  0  0]
 [ 2  0  3  0  0]
 [ 2  0  1  2  0]
 [ 0  0  0  1  1]]
[[ 0 10  2  0  0]
 [ 2  1  1  0  0]
 [ 2  0  0  3  0]
 [ 2  0  1  2  0]
 [ 0  0  0  1  1]]
[[ 0 10  2  0  0]
 [ 2  1  1  0  0]
 [ 2  0  0  0  0]
 [ 2  0  1 10  0]
 [ 0  0  0  4  0]]
________________
[[ 0 10  2  0  0]
 [ 2  1  1  0  0]
 [ 2  0  0  3  0]
 [ 2  0  1  2  0]
 [ 0  0  0  1  1]]
[[ 0 10  2  0  0]
 [ 2